# Snippets

## This file houses important bits and pieces of the code used (or not) in `utils.py`

Contains any resource that could prove helpful in later hotel projects when trying to remember how I did something.

In [1]:
import pandas as pd
import numpy as np
import datetime
import pickle

In [ ]:
# generte a list of dates from d1 to d2 (inclusive)

import datetime
d1 = datetime.date(2020, 1, 1)
d2 = datetime.date(2020, 4, 1)
all_dates = [datetime.datetime.strftime(d1 + datetime.timedelta(days=x), format="%Y-%m-%d") for x in range((d2-d1).days + 1)]